*importing libraries*

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import os
import datetime as dt

*Loading Dataset*

In [ ]:
try:
    online_retail = pd.read_csv(r"C:\Users\AdmiN\Desktop\RFM\Dataset\online_retail_II.csv")
    print("File loaded succesfully")
except FileNotFoundError:
    print("File not found")


File loaded succesfully


*Establishing connection with MySQL*

In [ ]:
USER = 'root'
PASSWORD = '8949762644'
HOST = 'localhost'
PORT = '3306'
DATABASE = 'retails'

# create connection engine

connection_string = f'mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}'
try:
    engine = create_engine(connection_string)
    print("Succefully connected to MySQL")
except Exception as e:
    print(f"Connection failed: {e}")
    raise                             

Succefully connected to MySQL


In [ ]:
# Changing column name for ease
online_retail = online_retail.rename(columns={'Customer ID': 'Customer_ID'})


*Loadiing data to MySQL*

In [ ]:
dataframes_to_load = {
    'online_retail': online_retail
}

try:
    for table_name, df in dataframes_to_load.items():
        print(f"🚀 Loading dataframe **{table_name}**...")
        
        df.to_sql(
            name=table_name,
            con=engine, 
            if_exists='replace',
            index=False,
            chunksize=1000 
            )
        print(f"✅ Done: {table_name}")
            
except SQLAlchemyError as e:
    print(f"❌ DATA LOAD FAILED: A transaction error occurred.")
    print(f"The transaction was automatically ROLLED BACK.")
    print(f"Error details: {e}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

print("Script finished.")

In [3]:
online_retail.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom


In [ ]:
online_retail.shape

In [ ]:
online_retail.info()

*Filtering data*

In [ ]:
# Removing rows where Quantity <= 0 AND Customer_ID is null AND price > 0
online_retail = pd.read_sql_query("""SELECT * FROM online_retail
                                  WHERE Customer_ID IS NOT NULL 
                                  AND Quantity > 0
                                  AND Price > 0""",engine)

In [6]:
# Ensuring the data has corrct data type
online_retail['Customer_ID'] = online_retail['Customer_ID'].astype(int)
online_retail['InvoiceDate'] = pd.to_datetime(online_retail['InvoiceDate'])

In [7]:
# Creating column total_Spend
online_retail['Total_Spend'] = online_retail['Quantity']*online_retail['Price']

In [8]:
# Creating new_table containing all customer
latest_date = online_retail['InvoiceDate'].max() + dt.timedelta(days=1)

new_table = online_retail.groupby('Customer_ID').agg({'InvoiceDate' : lambda x : (latest_date - x.max()).days,
                                                      'Invoice' : 'count',
                                                      'Total_Spend' : 'sum'}).reset_index()
# Rename columns
new_table.rename(columns={'InvoiceDate':'No_Of_Days',
                          'Invoice':'Invoice_Count'},inplace=True)             # Here,No_Of_Days is how many days it have to purchase an item

In [ ]:
new_table.head(10)

,Customer_ID,No_Of_Days,Invoice_Count,Total_Spend,Racency_Score,Frequency_Score,Spend_Score,Customer_Segment
0,12346,326,34,77556.46,1,1,4,Bronze / Inactive
1,12347,2,253,5633.32,4,3,4,Gold Member
2,12348,75,51,2019.40,3,2,3,Silver - New
3,12349,19,175,4428.69,4,3,3,Gold Member
4,12350,310,17,334.40,1,1,1,Bronze / Inactive
...,...,...,...,...,...,...,...,...
5873,18283,4,986,2736.65,4,4,3,Gold Member
5874,18284,432,28,461.68,1,1,1,Bronze / Inactive
5875,18285,661,12,427.00,1,1,1,Bronze / Inactive
5876,18286,477,67,1296.43,1,2,3,Bronze / Inactive


In [10]:
# seperating the data into four parts 
# we score from 1 - 4                     1 - lowscore(bad)   , 4 - Highscore(good)

# On basis of No_Of_days
# Here, we rank them if No_Of_Days is less than good else 'not good'            less days == high score

def score_recency(days):
    if days <= 30:                           # Bought in last month
        return 4
    elif days <= 90:                         # Bought in last Quarter
        return 3
    elif days <= 180:                        # Bought in half-year
        return 2
    else:                                    # Bought rarely
        return 1
    
# Score on basis of buying frequency (no of orders)              High count == High Score

def score_frequency(count):
    if count >= 300:
        return 4
    elif count >= 100:
        return 3
    elif count >= 50:
        return 2
    else:
        return 1
    
# Score on basis of total_spend                       High Spend == High Score

def  score_spend(amount):
    if amount >= 5000:
        return 4
    elif amount >= 1000:
        return 3
    elif amount >= 500:
        return 2
    else:
        return 1
    
# Creating columns on basis of scores

new_table['Racency_Score'] = new_table['No_Of_Days'].apply(score_recency)
new_table['Frequency_Score'] = new_table['Invoice_Count'].apply(score_frequency)
new_table['Spend_Score'] = new_table['Total_Spend'].apply(score_spend)


In [11]:
# Dividing customers into segments
def customer_segment(table):
    r_score = int(table['Racency_Score'])
    f_score = int(table['Frequency_Score'])
    s_score = int(table['Spend_Score'])

    # 1. THE BEST -> Platinum
    if r_score == 4 and f_score == 4 and s_score == 4:
        return 'Platinum Elite' 
    
    # 2. THE LOYAL -> Gold
    elif f_score >= 3 and s_score >= 3:
        return 'Gold Member' 
    
    # 3. THE NEW -> Silver
    elif r_score >= 3 and f_score <= 2:
        return 'Silver - New' 
    
    # 4. THE WORRISOME -> At Risk
    elif r_score <= 2 and f_score >= 3:
        return 'At Risk - Needs Promo' 
    
    # 5. THE LOWEST -> Bronze
    elif r_score <= 2 and f_score <= 2:
        return 'Bronze / Inactive' 
    
    # 6. EVERYONE ELSE
    else:
        return 'General Standard'
    

# Add column of customer_segment

new_table['Customer_Segment'] = new_table.apply(customer_segment,axis=1)

*Saving the final table to database retail for creating dashboad*

In [13]:
# Saving the final table to database retail
final_table = new_table

try:
    final_table.to_sql(name='final_table',con=engine,if_exists='replace',index=False)
    print("✅ Done")
except SQLAlchemyError as e:
    print("Failed")
    print(f"Error details : {e}")
except Exception as e:
    print(f"An unexpected Error occured : {e}")


✅ Done
